<a href="https://colab.research.google.com/github/sherirosalia/PPP_Covid_Stimulus/blob/master/random_forest/ML_allstates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

All States

 

In [ ]:
#import dependencies
import pandas as pd
import numpy as np
from datetime import date, datetime
import matplotlib
import os, sys 
import io
import ee


In [ ]:
## set up pandas dataframe using set_options to warn when we are working on a copy instead of ooriginal dataframe
pd.set_option('mode.chained_assignment','warn')

In [ ]:
# dataframe will display without being truncated
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns', 25)
pd.set_option('display.width', 1000)

In [ ]:
# !pip install -U -q PyDrive

In [ ]:
#mounting the google drive to access the files
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# command shows file path
# ! ls

In [ ]:
from google.colab import files

In [ ]:
# filepath
path = '/content/drive/My Drive/data/clean_ppp_data.csv'




In [ ]:
# Read column names from file
cols = list(pd.read_csv(path, nrows =1))
print(cols)

In [ ]:
# allowing an index column
ndf = pd.read_csv(path,  dtype={"Zip":str, "NAICS Code": str, "Jobs Saved":str }, parse_dates=[10], )
# df = pd.read_csv(path, error_bad_lines=False)
ndf.sample(3)

In [ ]:

ndf.dropna(axis='index',how='any',inplace=True)

In [ ]:
ndf['Jobs Saved'] = pd.to_numeric(ndf['Jobs Saved'])

In [ ]:
wf =  ndf['Bank']=='Wells Fargo Bank'
nd_wf=ndf[wf]
nd_wf.head()

In [ ]:
ndf.info()

In [ ]:
#check for nulls
print(ndf.isna().sum())


In [ ]:
#still have almost 4 million rows
ndf.shape

In [ ]:
#strip the decimal and zero
ndf['Jobs Saved'] = ndf['Jobs Saved'].astype(int)
ndf.sample()

In [ ]:
#take the first two letters of the NAICS code column and create category column. 
#category column can be matched to named values. The longer code was impractical
ndf['NAICS Category'] = ndf['NAICS Code'].str[:2]
ndf.info()

In [ ]:
# ndf['State'] = ndf['State'].str.rstrip('"', )

In [ ]:
# df['States'] = df['State'].str[:2]

# df.info()


In [ ]:
# df['States'] = df['States'].str.lstrip(' ', )

In [ ]:
ndf.head()

In [ ]:
ndf.sample(7)

In [ ]:
# ndf = ndf[ndf['Jobs Saved'] != '']
# df['NAICS Category'] = df['NAICS Category'].str.lstrip(' ', )

In [ ]:
ndf['State'].value_counts()

In [ ]:
ndf['Bank'].value_counts()

In [ ]:
# df['Bank'].where(df['Bank'].apply(lambda x:(x.value_counts()))>=500, "other")

In [ ]:
#curious about top bank counts

topbank=ndf['Bank'].value_counts()
topbank.head(25)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
cond = ndf['Bank'].value_counts()
threshold = 15000
ndf['Bank']= np.where(ndf['Bank'].isin(cond.index[cond >= threshold ]), ndf['Bank'], 'miscellaneous')

In [ ]:
top_banks_series=ndf.groupby('Bank', as_index=False).sum()
top_banks_series

In [ ]:
top_banks_series[0:-1].sort_index(ascending=False).plot(kind='barh',x='Bank', y='Loan Amount',)
figsize=(14,14)


left  = 0.125  # the left side of the subplots of the figure
right = 0.9    # the right side of the subplots of the figure
bottom = 0.1   # the bottom of the subplots of the figure
top = 0.9      # the top of the subplots of the figure
wspace = 0.2   # the amount of width reserved for blank space between subplots
hspace = 0.2   # the amount of height reserved for white space between subplots
# These two can be called on 'fig' instead of 'plt' too
plt.subplots_adjust(left=left, bottom=bottom, right=right, top=top,
                wspace=wspace, hspace=hspace)

plt.savefig('top_lenders.png',bbox_inches='tight')

##### running slow... try to fix plot later

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import tree
from sklearn.preprocessing import LabelEncoder

In [ ]:
ndf.sample()

In [ ]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()


In [ ]:
data=pd.DataFrame()
for x in ['Gender', 'Race', 'Veteran', 'State', 'Bank', 'Business Type']:
  label_encoder.fit(ndf[x])
  data[x] = label_encoder.transform(ndf[x])
data['Jobs Saved'] = ndf['Jobs Saved']

data['Loan Amount'] = ndf['Loan Amount']

In [ ]:
data.dtypes

In [ ]:
# data = data.fillna(lambda x: x.median())

In [ ]:
# data.fillna(data.mean(),inplace=True)

In [ ]:
# data.dtypes

In [ ]:
# np.any(np.isnan(X_train)) #true


In [ ]:
data.isnull().any()

In [ ]:
data.dropna(inplace=True)

In [ ]:
data.isnull().any()


In [ ]:
#split loan amounts off and drop from dataframe for ML
target=data['Loan Amount'] 
target.shape


In [ ]:
data=data.drop(['Loan Amount'],axis=1)
data.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)
rf = RandomForestRegressor(n_estimators=200, random_state=42)
rf.fit(X_train, y_train)

Score with: Gender | Race | Veteran | State | Bank | Jobs Saved | Business Type | Jobs Saved | NAICS

Score of forest = 0.381086540080108

Feature importance:
[0.02002078 0.01901149 0.01125769 0.08336426 0.20459831 0.0667973
 0.59495018]

In line with analysis on smaller datasets and the loans over 150K, the number of jobs saved carries the most weight on the amount of money lent.




In [ ]:
#score of forest
print(rf.score(X_test, y_test))



In [ ]:
#zip names and feature importance scores
f_imp=rf.feature_importances_
f_name=data.columns
features=zip(f_name, f_imp)
# print(features)
# features into a datafame
f_df=pd.DataFrame(list(features))
print(f_df)


In [ ]:
# X_train.columns
print(rf.feature_importances_)

In [ ]:
data.to_csv('small_loan_large_file.csv', index=0)
!cp people_ml.csv "drive/My Drive/"

In [ ]:
#convert the strings to integers in jobs saved
# df['Jobs Saved'] = pd.to_numeric(df['Jobs Saved'], errors="coerce")
# df['Jobs Saved'] = df['Jobs Saved'].astype(int)

In [ ]:
# df.to_csv('clean_ppp.csv', index=0)
# !cp clean_ppp.csv "drive/My Drive/"
